### Скрипт обработки объединенного файла открытого интереса с Московской биржи.

данные с 9:00 14 июня 2022 по 18:50 15 сентября 2022

*** данные сформированы по закрытию свечи, а нам нужно по открытию, поэтому вычитаем 5 минут ***

в последнее время Мосбиржа дает данные до 23-00 ( раньше данные были до 23-50)
нужно будет добавить с 23-00 до 23-50 данные по последнему состоянию


In [1]:
import pandas as pd
import datetime
from pathlib import Path

In [2]:
file_name = 'D:/Work/Data/OI_RIU2_conc.csv'

In [3]:
df = pd.read_csv (file_name, sep = ';', header = 0)

In [4]:
df

,sess_id,seqnum,tradedate,tradetime,ticker,clgroup,pos,pos_long,pos_short,pos_long_num,pos_short_num,systime
0,6576,105,2022-07-01,18:45:00,RI,YUR,24565,28695,-4130,25,34,2022-07-01 18:45:04
1,6576,105,2022-07-01,18:45:00,RI,FIZ,-24565,15265,-39830,1485,3545,2022-07-01 18:45:04
2,6576,104,2022-07-01,18:40:00,RI,YUR,24414,28682,-4268,25,34,2022-07-01 18:40:03
3,6576,104,2022-07-01,18:40:00,RI,FIZ,-24414,15513,-39927,1518,3573,2022-07-01 18:40:03
4,6576,103,2022-07-01,18:35:00,RI,YUR,24318,28792,-4474,28,32,2022-07-01 18:35:03
...,...,...,...,...,...,...,...,...,...,...,...,...
18123,6617,61,2022-08-29,10:15:00,RI,FIZ,-48424,27592,-76016,2090,5853,2022-08-29 10:15:04
18124,6617,60,2022-08-29,10:10:00,RI,YUR,48396,53234,-4838,51,33,2022-08-29 10:10:03
18125,6617,60,2022-08-29,10:10:00,RI,FIZ,-48389,27720,-76109,2046,5816,2022-08-29 10:10:03
18126,6617,59,2022-08-29,10:05:00,RI,YUR,48148,53073,-4925,49,33,2022-08-29 10:05:00


In [5]:
# Выбираем юрлица
yurl = df.loc[df.clgroup == 'YUR']

In [6]:
# Выбираем нужные столбцы, и systime ставим вперед
yurl_d = yurl[['tradedate', 'tradetime', 'pos_long', 'pos_short', 'pos_long_num', 'pos_short_num']]

In [7]:
# Переименовываем столбцы для yurl
yurl_d = yurl_d.rename(columns =
            {'pos_long' : 'yurl_l',
             'pos_short' : 'yurl_s',
             'pos_long_num' : 'yurl_l_num',
             'pos_short_num' : 'yurl_s_num'})            

In [8]:
# Делаем все тоже самое для fiz

# Выбираем физлица
fiz = df.loc[df.clgroup == 'FIZ']

# Выбираем нужные столбцы, и systime ставим вперед
fiz_d = fiz[['tradedate', 'tradetime', 'pos_long', 'pos_short', 'pos_long_num', 'pos_short_num']]

# Переименовываем столбцы для fiz
fiz_d = fiz_d.rename(columns =
            {'pos_long' : 'fiz_l',
             'pos_short' : 'fiz_s',
             'pos_long_num' : 'fiz_l_num',
             'pos_short_num' : 'fiz_s_num'})

# fiz_d.head()

In [9]:
# Соединяем оба df по yurl

oi = pd.merge(yurl_d, fiz_d, how='left')

In [10]:
# Удаляем дубликаты
oi = oi.drop_duplicates()

In [11]:
oi

,tradedate,tradetime,yurl_l,yurl_s,yurl_l_num,yurl_s_num,fiz_l,fiz_s,fiz_l_num,fiz_s_num
0,2022-07-01,18:45:00,28695,-4130,25,34,15265,-39830,1485,3545
1,2022-07-01,18:40:00,28682,-4268,25,34,15513,-39927,1518,3573
2,2022-07-01,18:35:00,28792,-4474,28,32,15670,-39988,1541,3586
3,2022-07-01,18:30:00,28898,-4585,30,31,15787,-40100,1561,3613
4,2022-07-01,18:25:00,29210,-4805,31,30,15817,-40222,1574,3635
...,...,...,...,...,...,...,...,...,...,...
9103,2022-08-29,10:25:00,53426,-4845,50,33,27408,-75989,2128,5910
9104,2022-08-29,10:20:00,53399,-4944,47,35,27550,-76006,2102,5886
9105,2022-08-29,10:15:00,53306,-4882,49,32,27592,-76016,2090,5853
9106,2022-08-29,10:10:00,53234,-4838,51,33,27720,-76109,2046,5816


In [12]:

# Добавляем новый столбец
mid = oi['tradedate'] + ' ' + oi['tradetime']
oi.insert(0, 'Trade_Time', mid)

In [13]:
# пока 'Trade_Time' строка - удаляем секунды ( последние 3 символа)
oi['Trade_Time'] = oi['Trade_Time'].str[:-3]

In [14]:
oi

,Trade_Time,tradedate,tradetime,yurl_l,yurl_s,yurl_l_num,yurl_s_num,fiz_l,fiz_s,fiz_l_num,fiz_s_num
0,2022-07-01 18:45,2022-07-01,18:45:00,28695,-4130,25,34,15265,-39830,1485,3545
1,2022-07-01 18:40,2022-07-01,18:40:00,28682,-4268,25,34,15513,-39927,1518,3573
2,2022-07-01 18:35,2022-07-01,18:35:00,28792,-4474,28,32,15670,-39988,1541,3586
3,2022-07-01 18:30,2022-07-01,18:30:00,28898,-4585,30,31,15787,-40100,1561,3613
4,2022-07-01 18:25,2022-07-01,18:25:00,29210,-4805,31,30,15817,-40222,1574,3635
...,...,...,...,...,...,...,...,...,...,...,...
9103,2022-08-29 10:25,2022-08-29,10:25:00,53426,-4845,50,33,27408,-75989,2128,5910
9104,2022-08-29 10:20,2022-08-29,10:20:00,53399,-4944,47,35,27550,-76006,2102,5886
9105,2022-08-29 10:15,2022-08-29,10:15:00,53306,-4882,49,32,27592,-76016,2090,5853
9106,2022-08-29 10:10,2022-08-29,10:10:00,53234,-4838,51,33,27720,-76109,2046,5816


In [15]:
# преобразуем systime в формат datetime
oi['Trade_Time']= pd.to_datetime(oi['Trade_Time'])

In [16]:
# Вычитаем 5 минут их systime

delta = pd.Timedelta(minutes = 5)
oi['Trade_Time'] = oi['Trade_Time'] - delta

In [17]:
oi

,Trade_Time,tradedate,tradetime,yurl_l,yurl_s,yurl_l_num,yurl_s_num,fiz_l,fiz_s,fiz_l_num,fiz_s_num
0,2022-07-01 18:40:00,2022-07-01,18:45:00,28695,-4130,25,34,15265,-39830,1485,3545
1,2022-07-01 18:35:00,2022-07-01,18:40:00,28682,-4268,25,34,15513,-39927,1518,3573
2,2022-07-01 18:30:00,2022-07-01,18:35:00,28792,-4474,28,32,15670,-39988,1541,3586
3,2022-07-01 18:25:00,2022-07-01,18:30:00,28898,-4585,30,31,15787,-40100,1561,3613
4,2022-07-01 18:20:00,2022-07-01,18:25:00,29210,-4805,31,30,15817,-40222,1574,3635
...,...,...,...,...,...,...,...,...,...,...,...
9103,2022-08-29 10:20:00,2022-08-29,10:25:00,53426,-4845,50,33,27408,-75989,2128,5910
9104,2022-08-29 10:15:00,2022-08-29,10:20:00,53399,-4944,47,35,27550,-76006,2102,5886
9105,2022-08-29 10:10:00,2022-08-29,10:15:00,53306,-4882,49,32,27592,-76016,2090,5853
9106,2022-08-29 10:05:00,2022-08-29,10:10:00,53234,-4838,51,33,27720,-76109,2046,5816


In [18]:
# Удаляем tradedate и tradetime
oi = oi.drop('tradedate', axis=1)
oi = oi.drop('tradetime', axis=1)
# отрицательные значения Sell  превращаем в положительные
oi['yurl_s'] = oi['yurl_s'] * -1
oi['fiz_s'] = oi['fiz_s'] * -1

In [19]:
oi

,Trade_Time,yurl_l,yurl_s,yurl_l_num,yurl_s_num,fiz_l,fiz_s,fiz_l_num,fiz_s_num
0,2022-07-01 18:40:00,28695,4130,25,34,15265,39830,1485,3545
1,2022-07-01 18:35:00,28682,4268,25,34,15513,39927,1518,3573
2,2022-07-01 18:30:00,28792,4474,28,32,15670,39988,1541,3586
3,2022-07-01 18:25:00,28898,4585,30,31,15787,40100,1561,3613
4,2022-07-01 18:20:00,29210,4805,31,30,15817,40222,1574,3635
...,...,...,...,...,...,...,...,...,...
9103,2022-08-29 10:20:00,53426,4845,50,33,27408,75989,2128,5910
9104,2022-08-29 10:15:00,53399,4944,47,35,27550,76006,2102,5886
9105,2022-08-29 10:10:00,53306,4882,49,32,27592,76016,2090,5853
9106,2022-08-29 10:05:00,53234,4838,51,33,27720,76109,2046,5816


In [20]:
# Сортировка по возрастанию ( день ближе к сегодня - последний)
oi = oi.sort_values(by='Trade_Time', ascending=True, na_position='first')

In [21]:
oi.head(10)

,Trade_Time,yurl_l,yurl_s,yurl_l_num,yurl_s_num,fiz_l,fiz_s,fiz_l_num,fiz_s_num
6983,2022-06-10 12:05:00,26201,5418,43,30,22091,42877,1740,4313
6982,2022-06-10 12:10:00,26282,5420,44,30,22095,42957,1737,4324
6981,2022-06-10 12:15:00,26234,5425,44,30,22221,43030,1749,4328
6980,2022-06-10 12:20:00,26327,5419,47,30,22310,43218,1763,4322
6979,2022-06-10 12:25:00,26484,5428,47,30,22516,43580,1755,4321
6978,2022-06-10 12:30:00,26568,5423,44,32,22326,43471,1716,4353
6977,2022-06-10 12:35:00,26684,5436,44,32,22375,43623,1711,4390
6976,2022-06-10 12:40:00,26769,5450,44,32,22373,43692,1729,4389
6975,2022-06-10 12:45:00,26866,5532,45,34,22653,43985,1734,4369
6974,2022-06-10 12:50:00,26956,5522,46,34,22648,44082,1736,4399


In [22]:
oi.tail()

,Trade_Time,yurl_l,yurl_s,yurl_l_num,yurl_s_num,fiz_l,fiz_s,fiz_l_num,fiz_s_num
5892,2022-12-09 22:35:00,36948,5534,45,49,27644,59058,2007,5416
5891,2022-12-09 22:40:00,36949,5533,45,49,27637,59053,2005,5416
5890,2022-12-09 22:45:00,36952,5532,45,49,27591,59011,1990,5418
5889,2022-12-09 22:50:00,36960,5532,46,49,27581,59009,1990,5416
5888,2022-12-09 22:55:00,36971,5533,45,48,27600,59038,1987,5419


In [23]:
# Сохраняем все в файл с добавлением _OI

oi.to_csv('D:/Work/Data/OI_RIU2.csv',
          index= False,
          sep = ";")